In [ ]:
!pip install opencv-python

In [1]:
import cv2
import numpy as np

In [3]:
video = cv2.VideoCapture("road_car_view.mp4")
while True:
    ret, or_frame = video.read()
    if not ret :
        video = cv2.VideoCapture("road_car_view.mp4")
        continue
    frame = cv2.GaussianBlur(or_frame, (5,5),0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_y = np.array([18,94,140])
    upper_y = np.array([48,255,255])
    
    mask = cv2.inRange(hsv, lower_y, upper_y)
    edges = cv2.Canny(mask, 74,150)
    lines = cv2.HoughLinesP(edges,1, np.pi/180, 50, maxLineGap = 50)
    
    if lines is not None :
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(frame, (x1,y1),(x2,y2),(0,255,0),5)
    cv2.imshow("frame", frame)
    cv2.imshow("edges", edges)
    key = cv2.waitKey(25)
    if(key==27):
        break
video.release()
cv2.destroyAllWindows()   
    

In [ ]:
!pip install PyAutoGUI

In [4]:
import pyautogui

In [5]:
image = pyautogui.screenshot()
image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
cv2.imwrite("my_testing_image.png", image)


True

#### We wanted to count the no. of vehicle and do the boundary box on it.
### Real Time case study

In [1]:
import cv2
import numpy as np
from time import sleep

In [5]:
count_line_pos = 550
min_width_rect = 80
min_height_rect = 80

offset = 6
delay = 60
detec = []
carros= 0


def central_handle(x,y,w,h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

cap = cv2.VideoCapture("video.mp4")
# Initialise subtractor algorithm
algo = cv2.bgsegm.createBackgroundSubtractorMOG()
while True:
    ret, frame1 = cap.read()
    #tempo = float(1/delay)
    #sleep(tempo)
    gray = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3),5)
    # Applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    countShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    
    cv2.line(frame1,(25,count_line_pos),(1200,count_line_pos),(0,0,255),3)
    
    for(i,c) in enumerate(countShape):
        (x,y,w,h) = cv2.boundingRect(c)
        validar_contorno = (w>=min_width_rect) and (h >=min_height_rect)
        if not validar_contorno:
            continue
        cv2.rectangle(frame1, (x,y),(x+w, y+h),(0,255,0),2)
        cv2.putText(frame1, "Vehicle"+str(carros), (x,y-20), cv2.FONT_HERSHEY_TRIPLEX,1,
                    (255,244,0),2)
        
               
        center = central_handle(x,y,w,h)
        detec.append(center)
        cv2.circle(frame1, center, 4,(0,0,255),-1 )
        
        # Loop Function
        
        for (x,y) in detec:
            if y<(count_line_pos+offset) and y>(count_line_pos-offset):
                carros +=1
                cv2.line(frame1, (25,count_line_pos), (1200, count_line_pos),(0,255,0), 3)
                detec.remove((x,y))
                print("Car is detected :" +str(carros) )
    cv2.putText(frame1, "Vehicle Count :" +str(carros), (450,70), cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,255),5)
        # https://www.rapidtables.com/web/color/RGB_Color.html
        
    #cv2.imshow("Detector", dilatada)    
    cv2.imshow("Video Original", frame1)
    cv2.imshow("Video Original", frame1)
    # cv2_imshow(frame1) # google colab
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()

Car is detected :1
Car is detected :2
Car is detected :3
Car is detected :4
Car is detected :5
Car is detected :6
Car is detected :7
Car is detected :8
Car is detected :9
Car is detected :10
Car is detected :11
Car is detected :12
Car is detected :13
Car is detected :14
Car is detected :15
Car is detected :16
Car is detected :17
Car is detected :18
Car is detected :19
Car is detected :20
Car is detected :21
Car is detected :22
Car is detected :23
Car is detected :24
Car is detected :25
Car is detected :26
Car is detected :27
Car is detected :28
Car is detected :29
Car is detected :30
Car is detected :31
Car is detected :32
Car is detected :33
Car is detected :34
Car is detected :35
Car is detected :36
Car is detected :37
Car is detected :38
Car is detected :39
Car is detected :40
Car is detected :41
Car is detected :42
Car is detected :43
Car is detected :44
Car is detected :45
Car is detected :46
Car is detected :47
Car is detected :48
Car is detected :49
Car is detected :50
Car is de

# Hand_Gesture

In [ ]:
!pip install mediapipe

In [6]:
import cv2
import numpy as np
import mediapipe as mp
import time

In [13]:
cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Frame per second
pTime = 0
cTime = 0

while True:
    ret, img = cap.read()
    imgBGR = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgBGR)
    print(results.multi_hand_landmarks)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            
            for id, lm in enumerate(handLms.landmark):
               # print(id, lm)
                
                h,w,c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                print(id, cx, cy)
                
                if id==4:
                    cv2.circle(img, (cx,cy),20, (255,0,255),cv2.FILLED)
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255),3)
    
    
    
    cv2.imshow('image', img)
    
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.23867730796337128
  y: 0.7668368816375732
  z: 7.276088922480994e-07
}
landmark {
  x: 0.3282244801521301
  y: 0.7873393297195435
  z: -0.05275052413344383
}
landmark {
  x: 0.42476484179496765
  y: 0.7345119118690491
  z: -0.07427766919136047
}
landmark {
  x: 0.5022590756416321
  y: 0.6783672571182251
  z: -0.08981068432331085
}
landmark {
  x: 0.5654725432395935
  y: 0.6360440850257874
  z: -0.1065690666437149
}
landmark {
  x: 0.4184226393699646
  y: 0.5595003962516785
  z: -0.05808435007929802
}
landmark {
  x: 0.4896245002746582
  y: 0.46170574426651
  z: -0.09376763552427292
}
landmark {
  x: 0.536533772945404
  y: 0.3956300914287567
  z: -0.12004983425140381
}
landmark {
  x: 0.5753541588783264
  y: 0.33919137716293335
  z: -0.1394689977169037
}
landmark {
  x: 0.3665265738964081
  y: 0.5046238303184509
  z: -0.056711845099925995
}
landmark {
  x: 0.4225066304206848
  y: 0.382967680692

[landmark {
  x: 0.31769296526908875
  y: 0.545767068862915
  z: 7.443962317665864e-07
}
landmark {
  x: 0.4088476896286011
  y: 0.5434447526931763
  z: -0.0423625148832798
}
landmark {
  x: 0.495471715927124
  y: 0.48622000217437744
  z: -0.05868111923336983
}
landmark {
  x: 0.565046489238739
  y: 0.4204923212528229
  z: -0.07347816973924637
}
landmark {
  x: 0.630124032497406
  y: 0.3934263288974762
  z: -0.08668294548988342
}
landmark {
  x: 0.5113038420677185
  y: 0.313232421875
  z: -0.011928635649383068
}
landmark {
  x: 0.572382390499115
  y: 0.225498765707016
  z: -0.03454890474677086
}
landmark {
  x: 0.6105408668518066
  y: 0.16511377692222595
  z: -0.056715093553066254
}
landmark {
  x: 0.6414718627929688
  y: 0.11531181633472443
  z: -0.0736355409026146
}
landmark {
  x: 0.464307963848114
  y: 0.2616896629333496
  z: -0.011963693425059319
}
landmark {
  x: 0.5142976641654968
  y: 0.14079803228378296
  z: -0.031046761199831963
}
landmark {
  x: 0.5482354760169983
  y: 0.062

[landmark {
  x: 0.3427637815475464
  y: 0.5510551333427429
  z: 5.38122549187392e-07
}
landmark {
  x: 0.427278995513916
  y: 0.5458675622940063
  z: -0.03981301560997963
}
landmark {
  x: 0.512978196144104
  y: 0.48689255118370056
  z: -0.054228685796260834
}
landmark {
  x: 0.5818896293640137
  y: 0.4261854588985443
  z: -0.06691593676805496
}
landmark {
  x: 0.6409369111061096
  y: 0.3955450654029846
  z: -0.07868883013725281
}
landmark {
  x: 0.5264831781387329
  y: 0.3256763815879822
  z: -0.016772475093603134
}
landmark {
  x: 0.5840620994567871
  y: 0.24182450771331787
  z: -0.039547938853502274
}
landmark {
  x: 0.620086669921875
  y: 0.18448466062545776
  z: -0.061618298292160034
}
landmark {
  x: 0.6509580612182617
  y: 0.13575318455696106
  z: -0.0785892903804779
}
landmark {
  x: 0.4837929308414459
  y: 0.27606961131095886
  z: -0.017013410106301308
}
landmark {
  x: 0.5302930474281311
  y: 0.1597362905740738
  z: -0.03543505445122719
}
landmark {
  x: 0.5628972053527832
 

[landmark {
  x: 0.3550650477409363
  y: 0.5587970018386841
  z: 5.819520083605312e-07
}
landmark {
  x: 0.43869078159332275
  y: 0.5563033819198608
  z: -0.04182695969939232
}
landmark {
  x: 0.5221090316772461
  y: 0.5031676292419434
  z: -0.05952823907136917
}
landmark {
  x: 0.5882945656776428
  y: 0.44268304109573364
  z: -0.07461328059434891
}
landmark {
  x: 0.6485572457313538
  y: 0.4103664755821228
  z: -0.08864089101552963
}
landmark {
  x: 0.5310561060905457
  y: 0.3428448438644409
  z: -0.023213449865579605
}
landmark {
  x: 0.5886120200157166
  y: 0.2603583037853241
  z: -0.04699781909584999
}
landmark {
  x: 0.6234991550445557
  y: 0.20490001142024994
  z: -0.06809155642986298
}
landmark {
  x: 0.6534445285797119
  y: 0.15675349533557892
  z: -0.08453182876110077
}
landmark {
  x: 0.4892446994781494
  y: 0.2954230308532715
  z: -0.023129548877477646
}
landmark {
  x: 0.5348518490791321
  y: 0.1840767115354538
  z: -0.04418245702981949
}
landmark {
  x: 0.5667387247085571


0 615 313
1 566 305
2 523 272
3 494 239
4 464 220
5 536 193
6 514 145
7 502 114
8 495 87
9 570 179
10 557 118
11 551 80
12 547 50
13 604 178
14 599 122
15 595 87
16 591 57
17 639 188
18 646 147
19 652 120
20 657 96
[landmark {
  x: 0.3870925307273865
  y: 0.6003282070159912
  z: 5.31562704964017e-07
}
landmark {
  x: 0.4708413779735565
  y: 0.5905063152313232
  z: -0.03452824801206589
}
landmark {
  x: 0.5527759790420532
  y: 0.5400755405426025
  z: -0.05058186128735542
}
landmark {
  x: 0.617658257484436
  y: 0.49043571949005127
  z: -0.06542587280273438
}
landmark {
  x: 0.6751765012741089
  y: 0.46610304713249207
  z: -0.07954729348421097
}
landmark {
  x: 0.5512123703956604
  y: 0.3783319890499115
  z: -0.019544998183846474
}
landmark {
  x: 0.6055241823196411
  y: 0.2931475043296814
  z: -0.042813561856746674
}
landmark {
  x: 0.6389168500900269
  y: 0.2404015064239502
  z: -0.06403443962335587
}
landmark {
  x: 0.6694520711898804
  y: 0.19423739612102509
  z: -0.08013981580734253

...

[landmark {
  x: 1.0265507698059082
  y: 0.6851330995559692
  z: 7.151982117648004e-07
}
landmark {
  x: 0.9468637108802795
  y: 0.6703786849975586
  z: -0.05432002246379852
}
landmark {
  x: 0.8781787157058716
  y: 0.6006120443344116
  z: -0.06872145086526871
}
landmark {
  x: 0.842121958732605
  y: 0.523386538028717
  z: -0.07490618526935577
}
landmark {
  x: 0.815401017665863
  y: 0.4497632682323456
  z: -0.07793686538934708
}
landmark {
  x: 0.9148945212364197
  y: 0.4860273003578186
  z: -0.010623691603541374
}
landmark {
  x: 0.8983145952224731
  y: 0.3938602805137634
  z: -0.020789029076695442
}
landmark {
  x: 0.891787052154541
  y: 0.3265351355075836
  z: -0.03516332060098648
}
landmark {
  x: 0.8896477818489075
  y: 0.2717049717903137
  z: -0.04573281854391098
}
landmark {
  x: 0.9619386196136475
  y: 0.46304935216903687
  z: 0.008661648258566856
}
landmark {
  x: 0.9497387409210205
  y: 0.37380844354629517
  z: 0.006718480493873358
}
landmark {
  x: 0.9393880367279053
  y: 0

[landmark {
  x: 1.0406454801559448
  y: 0.7143734693527222
  z: 1.0086858992508496e-06
}
landmark {
  x: 0.9780696034431458
  y: 0.6948058009147644
  z: -0.09404740482568741
}
landmark {
  x: 0.9331085681915283
  y: 0.6241971850395203
  z: -0.1488030105829239
}
landmark {
  x: 0.900001049041748
  y: 0.543144166469574
  z: -0.18610019981861115
}
landmark {
  x: 0.8533151149749756
  y: 0.4700433909893036
  z: -0.2234763503074646
}
landmark {
  x: 0.9849856495857239
  y: 0.4966427683830261
  z: -0.13647176325321198
}
landmark {
  x: 0.9653570652008057
  y: 0.36971426010131836
  z: -0.1899699717760086
}
landmark {
  x: 0.958522617816925
  y: 0.2710409462451935
  z: -0.2257230430841446
}
landmark {
  x: 0.9568160772323608
  y: 0.18978983163833618
  z: -0.247503399848938
}
landmark {
  x: 1.0072383880615234
  y: 0.48109206557273865
  z: -0.10479152202606201
}
landmark {
  x: 0.984468400478363
  y: 0.3546559810638428
  z: -0.15568429231643677
}
landmark {
  x: 0.9730547070503235
  y: 0.24946

[landmark {
  x: 0.32037392258644104
  y: 0.9476761817932129
  z: 5.646286922456056e-07
}
landmark {
  x: 0.3938625752925873
  y: 0.843957245349884
  z: -0.02589091844856739
}
landmark {
  x: 0.43995827436447144
  y: 0.713040292263031
  z: -0.04745393246412277
}
landmark {
  x: 0.4781845211982727
  y: 0.6190152168273926
  z: -0.07079301029443741
}
landmark {
  x: 0.5205009579658508
  y: 0.5470679402351379
  z: -0.09677761048078537
}
landmark {
  x: 0.3463684320449829
  y: 0.5635054111480713
  z: -0.04078684374690056
}
landmark {
  x: 0.3415150046348572
  y: 0.41899746656417847
  z: -0.08097681403160095
}
landmark {
  x: 0.344703733921051
  y: 0.3359212279319763
  z: -0.1123373731970787
}
landmark {
  x: 0.3506145477294922
  y: 0.25793182849884033
  z: -0.13660500943660736
}
landmark {
  x: 0.2996247410774231
  y: 0.5797112584114075
  z: -0.06116638332605362
}
landmark {
  x: 0.2772044539451599
  y: 0.4217625856399536
  z: -0.10108112543821335
}
landmark {
  x: 0.2717224657535553
  y: 0

[landmark {
  x: 0.3553125262260437
  y: 0.9484717845916748
  z: 6.50964068427129e-07
}
landmark {
  x: 0.4323078691959381
  y: 0.8581342697143555
  z: -0.028358928859233856
}
landmark {
  x: 0.4738262891769409
  y: 0.7202305197715759
  z: -0.05353034660220146
}
landmark {
  x: 0.5204958915710449
  y: 0.628751814365387
  z: -0.0799117237329483
}
landmark {
  x: 0.5728759169578552
  y: 0.5657868981361389
  z: -0.10924053937196732
}
landmark {
  x: 0.3776378631591797
  y: 0.5470435619354248
  z: -0.04824845492839813
}
landmark {
  x: 0.37491536140441895
  y: 0.4031168818473816
  z: -0.09407981485128403
}
landmark {
  x: 0.3866451382637024
  y: 0.3174455463886261
  z: -0.12968707084655762
}
landmark {
  x: 0.4023737609386444
  y: 0.23855265974998474
  z: -0.15646547079086304
}
landmark {
  x: 0.33456915616989136
  y: 0.568369448184967
  z: -0.06869600713253021
}
landmark {
  x: 0.3240610659122467
  y: 0.4097210168838501
  z: -0.11122247576713562
}
landmark {
  x: 0.3387484848499298
  y: 0

[landmark {
  x: 0.4243476688861847
  y: 1.0100429058074951
  z: 7.375818427135528e-07
}
landmark {
  x: 0.5049647688865662
  y: 0.9149126410484314
  z: -0.031798332929611206
}
landmark {
  x: 0.5647450089454651
  y: 0.7830807566642761
  z: -0.053228601813316345
}
landmark {
  x: 0.6178560853004456
  y: 0.6927950382232666
  z: -0.07510799169540405
}
landmark {
  x: 0.6634325385093689
  y: 0.6190082430839539
  z: -0.09792042523622513
}
landmark {
  x: 0.4684838652610779
  y: 0.5938608050346375
  z: -0.02888273075222969
}
landmark {
  x: 0.4717549681663513
  y: 0.45164602994918823
  z: -0.06665940582752228
}
landmark {
  x: 0.48540589213371277
  y: 0.3721677362918854
  z: -0.10006952285766602
}
landmark {
  x: 0.5058375597000122
  y: 0.2991761267185211
  z: -0.1252146065235138
}
landmark {
  x: 0.42222005128860474
  y: 0.606687605381012
  z: -0.042311470955610275
}
landmark {
  x: 0.4225403368473053
  y: 0.45479875802993774
  z: -0.07258924841880798
}
landmark {
  x: 0.4412998855113983
 